In [1]:
import sys
sys.path.append('./yolo')
from detection_fusion import EnsembleModel
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from rcnn.dataset import ChestCocoDetection
from torchvision import transforms
import matplotlib.patches as patches
from torch.utils.data import DataLoader
import tqdm

In [2]:
def add_bounding_boxes(target, ax):
    boxes = target['boxes']
    for box in boxes:
        mp_box = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], edgecolor="r", facecolor='none')
        ax.add_patch(mp_box)

def show_samples_for(test, train):
    fig, axs = plt.subplots(1, 2, figsize=(10,8))
    #fig.suptitle(f'Study: {samples["id"].iloc[i]}')
    for im_i, (img, data) in enumerate([test, train]):
        ax = axs[im_i] if isinstance(axs, np.ndarray) else axs
        ax.set_title(f'Instance / Image Nr. {im_i + 1} / {2}')
        #dcm = pydicom.dcmread(image_path)
        plt.figure()
        ax.imshow(img.permute(1,2,0))#,cmap=plt.cm.bone)
        add_bounding_boxes(data, ax)
    fig.savefig("combo.jpg")

In [ ]:
test = ChestCocoDetection(root="F:\\aml-project\data\\siim-covid19-detection", ann_file="F:\\aml-project\\data\\siim-covid19-detection\\test.json", training=False)

In [ ]:
idx = np.random.randint(len(test))
test_img = test.__getitem__(idx)
print(type(test_img[0]), test_img[0].shape)
print(test_img[1])
img = test_img[0]
img_pil = transforms.ToPILImage()(img)

In [ ]:
results, boxes, scores = detection_fusion(img_pil)

In [ ]:
show_samples_for(test_img, (test_img[0], {"boxes":boxes}))

### Evaluation of Ensemble method

In [8]:
from yolo.utils.general import xywh2xyxy, scale_coords, box_iou
from yolo.utils.metrics import ap_per_class
import torch
from rcnn.model import ChestRCNN
from yolo.yolo import Model

In [9]:
def collate_fn(batch):
    imgs, targets = zip(*batch)
    imgs = torch.stack(imgs)
    return tuple((imgs, targets))

In [10]:
# need bs = 1 because model fusion cannot handle batches 
test = ChestCocoDetection(root="F:\\aml-project\data\\siim-covid19-detection", ann_file="F:\\aml-project\\data\\siim-covid19-detection\\test.json", training=False)
test_loader = DataLoader(test, batch_size=1, shuffle=False, pin_memory=False, num_workers=0, collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [11]:
YOLO_FINAL_MODEL_PATH = "./yolo/models_final_giou_40/yolov5_epoch_25.pt"
FASTER_RCNN_FINAL_MODEL_PATH = "./rcnn/models/fasterrcnn_epoch_23.pt"
RESNET_BACKBONE_PATH = "./resnet/models/resnext101_32x8d_epoch_35.pt"

yolov5_weights = torch.load(YOLO_FINAL_MODEL_PATH)
fasterrcnn_r101_weights = torch.load(FASTER_RCNN_FINAL_MODEL_PATH)

yolo = Model(cfg="./yolo/yolo5l.yaml",ch=3,nc=1)
yolo.load_state_dict(yolov5_weights, strict=False) 

fasterRcnn = ChestRCNN(RESNET_BACKBONE_PATH)
fasterRcnn.load_state_dict(fasterrcnn_r101_weights)
ensemble = EnsembleModel(fasterRcnn=fasterRcnn, yolo=yolo)

D:\Anaconda\envs\aml\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
jdict, stats, ap, ap_class = [], [], [], []
iouv = torch.linspace(0.5, 0.55, 1) # iou vector for mAP@0.5:0.95
niou = iouv.numel()
general_test_results = {}
for i, (images, targets) in enumerate(tqdm.tqdm(test_loader)):
    nb, _, height, width = images.shape
    assert nb == 1
    img_pil = transforms.ToPILImage()(images[0])
    _, boxes, scores, pred_labels, _, _, _, _ = ensemble.detection_fusion(img_pil, extended_output=True)
    boxes = torch.tensor(boxes)
    gt_boxes = targets[0]['boxes']
    labels = targets[0]['labels']
    nl = len(labels)
    tcls = labels[:].tolist() if nl else []  # target class

    if len(boxes) == 0:
        if nl:
            stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
        continue

    correct = torch.zeros(len(boxes), niou, dtype=torch.bool)
    if nl:
        detected = []
        tcls_tensor = labels[:]

        # Per target class
        for cls in torch.tensor([1]):
            ti = (cls == tcls_tensor).nonzero(as_tuple=False).view(-1)
            pi = (cls == torch.tensor(pred_labels)).nonzero(as_tuple=False).view(-1) 

            # Search for detections
            if pi.shape[0]:
                ious, i = box_iou(boxes[pi], gt_boxes[ti]).max(1)  # best ious, indices

                # Append detections
                detected_set = set()
                for j in (ious > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i[j]]  
                    if d.item() not in detected_set:
                        detected_set.add(d.item())
                        detected.append(d)
                        correct[pi[j]] = ious[j] > iouv  
                        if len(detected) == nl: 
                            break

        # Append statistics (correct, conf, pcls, tcls)
        #print((correct.cpu(), torch.tensor(scores),torch.tensor(pred_labels), tcls))
        stats.append((correct.cpu(), torch.tensor(scores),torch.tensor(pred_labels), tcls))

stats = [np.concatenate(x, 0) for x in zip(*stats)] 
if len(stats) and stats[0].any():
    p, r, ap, f1, ap_class = ap_per_class(*stats)
    ap50, ap = ap[:, 0], ap.mean(1) 
    mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
    nt = np.bincount(stats[3].astype(np.int64), minlength=1)

    general_test_results.append({
    "precision": p,
    "recall": r,
    "ap": ap,
    "f1": f1,
    "ap_class": ap_class,
    "ap": ap,
    "ap50": ap50,
    "mp": mp,
    "mr": mr,
    "map50": map50,
    "map": map
})

else:
    nt = torch.zeros(1)

# Print results
pf = '%20s' + '%12i' * 2 + '%12.3g' * 4 
print(pf % ('all', seen, nt.sum(), mp, mr, map50, map))

  5%|███▊                                                                           | 41/859 [06:17<2:02:16,  8.97s/it]

In [ ]:
map50